# Explore here

In [236]:
import pandas as pd 
import ast
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 


importamos pandas y leemos los data sets iniciales

In [237]:
peliculas = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
creditos = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')

In [264]:
data_sin_procesar = pd.concat([peliculas, creditos])

In [265]:
data_sin_procesar.to_csv(rf'C:\Users\wipip\OneDrive\Documentos\GitHub\KNN-Project-main\data\raw\data_sin_procesar.csv')

Empezamos importando sqalchemy y creando una base de datos utilizando SQLite

In [238]:
peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [239]:
creditos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [240]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:')

Es importante mencionar que en este punto utilizamos SQLite por la simplicidad, pero podriamos utlizar MySQL o postgreSQL, sin embargo esto comunmente requiere de configuraciones adicionales, por lo que continuaremos con esta ruta. Lo unico que nos puede ser contraproducente es que la base de datos que acabamos de crear esta disponible unicamente mientras ejecutamos el programa.

Ahora lo que haremos sera guardar los data frames que acabamos de crear en tablas de sql para poder manipularlos dentro de la base de datos. 

In [241]:
peliculas.to_sql('peliculas', engine, index=False)
creditos.to_sql('creditos', engine, index=False) 


4803

Ahora vamos a unir las tablas utilizando SQL, utilizaremos querys para escribir nuestro codigo de sql y posteriormente utilizaremos la funcion de read sql para transformar nuestra tabla de sql de nuevo en un data frame de pandas para comenzar a hacer nuesto EDA. 


In [242]:
query = """ 
SELECT creditos.movie_id, peliculas.title, peliculas.overview, peliculas.genres, peliculas.keywords, creditos.cast, creditos.crew
FROM peliculas
JOIN creditos 
ON peliculas.title = creditos.title
"""

data = pd.read_sql(query, engine)

En este caso estaremos utilizando el query, que es el nombre comun que se le da a las cadenas de consulta de SQL, la funcion especifica que creamos con el query es la de seleccionar columnas especificas de las tablas de peliculas y creditos para unirlas usando como clave de  union el titulo de las peliculas. En otras palabras, hacemos el query para seleccionar columnas especificas de nuestras tablas donde los titulos coinciden.

Lo siguiente que haremos sera crear un data frame llamado " data " en el cual vamos a asignar la  ejecucion de la consulta en SQL a la base de datos.
 

In [243]:
data.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Ahora tenemos que transformar los datos debido a las columnas que tienen un formato JSON

In [244]:
def obtener_nombres(columna):
    return [i['name'] for i in ast.literal_eval(columna)]

data['genres'] = data['genres'].apply(obtener_nombres)
data['keywords'] = data['keywords'].apply(obtener_nombres)
data['cast'] = data['cast'].apply(lambda x: obtener_nombres(x)[:3])

In [245]:
def seleccionar_director(columna):
    for i in ast.literal_eval(columna):
        if i['job'] == 'Director':
            return i['name']
    return np.nan

data['crew']= data['crew'].apply(seleccionar_director)

In [246]:
data.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [247]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4779 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [248]:
data.dropna(inplace=True)

In [249]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4776 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4776 non-null   int64 
 1   title     4776 non-null   object
 2   overview  4776 non-null   object
 3   genres    4776 non-null   object
 4   keywords  4776 non-null   object
 5   cast      4776 non-null   object
 6   crew      4776 non-null   object
dtypes: int64(1), object(6)
memory usage: 298.5+ KB


Ahora vamos a eliminar los espacios de las palabras, convertir las listas a strings y eliminar los corchetes y comiilas simples

In [250]:
import re
def eliminar_espacio(palabra):
    str_limpio = re.sub(r'\s+', '', palabra).strip()
    return str_limpio

In [251]:

data['crew'] = data['crew'].apply(eliminar_espacio)
data['crew'][0]

'JamesCameron'

In [252]:
data['cast'] = data['cast'].apply(lambda x: [''.join(i.split()) for i in x])
data['genres'] = data['genres'].apply(lambda x: [''.join(i.split()) for i in x])
data['keywords'] = data['keywords'].apply(lambda x: [''.join(i.split()) for i in x])

In [253]:
print(data['cast'][0],
data['genres'][0],
data['crew'][0],
data['keywords'][0])

['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver'] ['Action', 'Adventure', 'Fantasy', 'ScienceFiction'] JamesCameron ['cultureclash', 'future', 'spacewar', 'spacecolony', 'society', 'spacetravel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alienplanet', 'cgi', 'marine', 'soldier', 'battle', 'loveaffair', 'antiwar', 'powerrelations', 'mindandsoul', '3d']


In [254]:
data['cast'] = data['cast'].astype(str)
data['genres'] = data['genres'].astype(str)
data['keywords'] = data['keywords'].astype(str)

In [255]:
data['tags'] = data['genres'] + ' ' + data['keywords'] + ' ' + data['cast'] + ' ' + data['crew']

In [256]:
def quitar_puntuacion(columna):
    columna = columna.replace("'", "")
    columna = columna.replace("]", "")
    columna = columna.replace("[", "")
    columna = columna.replace(",", "")
    return columna

In [257]:
data['tags'] = data['tags'].apply(quitar_puntuacion)

In [258]:
data['tags'] = data['overview'] + ' ' + data['tags']

In [259]:
data['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

Finalmente convertimos el overview en una lista

In [260]:
data['overview'] = data['overview'].apply(lambda x: x.split())

In [261]:
data.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","['Action', 'Adventure', 'Fantasy', 'ScienceFic...","['cultureclash', 'future', 'spacewar', 'spacec...","['SamWorthington', 'ZoeSaldana', 'SigourneyWea...",JamesCameron,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","['Adventure', 'Fantasy', 'Action']","['ocean', 'drugabuse', 'exoticisland', 'eastin...","['JohnnyDepp', 'OrlandoBloom', 'KeiraKnightley']",GoreVerbinski,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","['Action', 'Adventure', 'Crime']","['spy', 'basedonnovel', 'secretagent', 'sequel...","['DanielCraig', 'ChristophWaltz', 'LéaSeydoux']",SamMendes,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","['Action', 'Crime', 'Drama', 'Thriller']","['dccomics', 'crimefighter', 'terrorist', 'sec...","['ChristianBale', 'MichaelCaine', 'GaryOldman']",ChristopherNolan,Following the death of District Attorney Harve...
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","['Action', 'Adventure', 'ScienceFiction']","['basedonnovel', 'mars', 'medallion', 'spacetr...","['TaylorKitsch', 'LynnCollins', 'SamanthaMorton']",AndrewStanton,"John Carter is a war-weary, former military ca..."


Una vez que transformamos los datos y creamos la columna de tags para vectorizacion, daremos por terminada la fase de exploracion, guardaremos el datset en un archivo csv y continuaremos en app.ipynb con la implementacion del modelo. 

In [262]:
data_interina = data

data_interina.to_csv(rf'C:\Users\wipip\OneDrive\Documentos\GitHub\KNN-Project-main\data\interim\data_con_tags.csv', index=False)